<a href="https://colab.research.google.com/github/mea2220/Interactive-Visualization-Lovable-Report/blob/main/Assignment_4_Alexander.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Assignment 4 :: PySpark Data Queries

##### **Install PySpark (Python Spark API)**


In [3]:
!pip install -U pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.2/434.2 MB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.0/203.0 kB 17.9 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-4.0.1-py2.py3-none-any.whl size=434813800 sha256=5a89290daf8692ca16619edc10600b7b72791b4237e8f4bf630bd8106b05f4f8
  Stored in directory: /root/.cache/pip/wheels/31/9f/68/f89fb34ccd886909be7d0e390eaaf97f21efdf540c0ee8dbcd
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7
  Attempting uninstall: pyspark
    Found existing installation: pyspark 3.5.1
    Uninstalling pyspark-3.5.1:
      Successfully uninstalled pyspark-3.5.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dataproc-spark-conn

In [4]:
import os
import sys
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [5]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Intro to Apache Spark") \
    .config("spark.cores.max", "4") \
    .config('spark.executor.memory', '8G') \
    .config('spark.driver.maxResultSize', '8g') \
    .config('spark.kryoserializer.buffer.max', '512m') \
    .config("spark.driver.cores", "4") \
    .getOrCreate()

sc = spark.sparkContext

print("Using Apache Spark Version", spark.version)

Using Apache Spark Version 4.0.1


##### **Ingest Data & Count Rows**


In [8]:
#Read the Crunchbase ODM Orgs CSV into a Spark DataFrame
cb_sdf = spark.read.format("csv") \
               .options(header='true', inferschema='true', treatEmptyValuesAsNulls='true') \
               .load("/content/crunchbase_odm_orgs.csv")

#Print the count of all records
cb_sdf.count()

1127735

In [13]:
#see columns
cb_sdf.columns

['uuid',
 'name',
 'type',
 'primary_role',
 'cb_url',
 'domain',
 'homepage_url',
 'logo_url',
 'facebook_url',
 'twitter_url',
 'linkedin_url',
 'combined_stock_symbols',
 'city',
 'region',
 'country_code',
 'short_description']

##### **A) Find all companies with names that contain exactly three words  (e.g. "Fox Interactive Media")**

In [11]:
##Print the count of such companies

import pyspark.sql.functions as F

three_word_companies = cb_sdf.filter(F.size(F.split(F.col("name"), " ")) == 3)
three_word_companies.count()

206299

In [12]:
##Use show() to display only the name and location (city, region, country_code) in the resulting Spark DataFrame

three_word_companies.select("name", "city", "region", "country_code").show()

+--------------------+-------------+----------------+------------+
|                name|         city|          region|country_code|
+--------------------+-------------+----------------+------------+
|Fox Interactive M...|Beverly Hills|      California|         USA|
|Hutchison Whampoa...|    Hong Kong|Hong Kong Island|         HKG|
|      Gannett Co Inc|       Mclean|        Virginia|         USA|
|          HOT or NOT|San Francisco|      California|         USA|
|        Funny Or Die|    San Mateo|      California|         USA|
|Peak Steal Buildings|       Morgan|         Georgia|         USA|
|      Kosiso - store|      Chicago|        Illinois|         USA|
|The Accelerator G...|       Mclean|        Virginia|         USA|
|    Real Time Matrix|      Oakland|      California|         USA|
|  Keep Highways Safe|   Jenkintown|    Pennsylvania|         USA|
|    Red Light Center|         NULL|            NULL|        NULL|
|    Small World Labs|       Austin|           Texas|         

##### **B) Find all companies located in the state of New Jersey:**

In [14]:
##Print the count of such companies

new_jersey_companies = cb_sdf.where((F.col('region') == "New Jersey"))

new_jersey_companies.count()

10251

In [15]:
##Use show() to display only the name and location (city, region, country_code) in the resulting Spark DataFrame
new_jersey_companies.select("name", "city", "region", "country_code").show()

+-------------------+--------------+----------+------------+
|               name|          city|    region|country_code|
+-------------------+--------------+----------+------------+
|           Blogsigs|       Madison|New Jersey|         USA|
|              MSNBC|      Secaucus|New Jersey|         USA|
|            BitWine|       Tenafly|New Jersey|         USA|
|          GoAmerica|    Hackensack|New Jersey|         USA|
|          Phone.com|        Newark|New Jersey|         USA|
|           Phanfare|     Princeton|New Jersey|         USA|
|              Vidyo|    Hackensack|New Jersey|         USA|
|          eventsbot|    Piscataway|New Jersey|         USA|
|       CheerOutLoud|   Perth Amboy|New Jersey|         USA|
|        Jet Numbers|East Brunswick|New Jersey|         USA|
|       ShareMethods|  South Orange|New Jersey|         USA|
|      Winescorecard|  Lambertville|New Jersey|         USA|
|              ePrep|     Princeton|New Jersey|         USA|
|     Origin Digital|   

##### **C) Add a "TechDomain" column to the DataFrame where the row entries are set to 1 if the "domain" field contains "github.io", and 0 otherwise:**

In [16]:
cb_sdf_c = cb_sdf.withColumn(
    'TechDomain',
    F.when(F.col('domain').like('%github.io%'), 1).otherwise(0)
)

##Use show() to display only the name, location (city, region, country_code) and TechDomain column for companies where the TechDomain field is marked as 1.
cb_sdf_c.filter(F.col('TechDomain') == 1).select('name', 'city', 'region', 'country_code', 'TechDomain').show()

+--------------------+-------------+------------+------------+----------+
|                name|         city|      region|country_code|TechDomain|
+--------------------+-------------+------------+------------+----------+
|               OSSEC|San Francisco|  California|         USA|         1|
|            Fosstrak|San Francisco|  California|         USA|         1|
|       littlesnapper|         NULL|        NULL|        NULL|         1|
|  Associação CRIArte|         NULL|        NULL|        NULL|         1|
|           Bros Labs|        Hanoi|NA - Vietnam|         VNM|         1|
|             Telcial|      Brenham|       Texas|         USA|         1|
|      Dev Free Casts|         NULL|        NULL|        NULL|         1|
|  Ojas Open Platform|San Francisco|  California|         USA|         1|
|       Bababa Studio|San Francisco|  California|         USA|         1|
|             Nova IO|San Francisco|  California|         USA|         1|
|Zest - Discover E...|San Francisco|  

##### **D) Find all companies whose names contain at least one repeating consecutive letter (e.g., "Google", "Twitter", "Massa Inc.") using a Spark UDF function.**

In [19]:
from pyspark.sql.functions import udf, col
from pyspark.sql.types import BooleanType
import re

# Define a Python function to check for repeating consecutive letters
def has_repeating_letters(name):
    if name is None:
        return False
    return bool(re.search(r'(.)\1', name, re.IGNORECASE))

# Register the Python function as a Spark UDF
repeating_letters_udf = udf(has_repeating_letters, BooleanType())

# Apply the UDF to the DataFrame and filter
repeating_letter_companies = cb_sdf.filter(repeating_letters_udf(col("name")))

##Print the count of such companies
repeating_letter_companies.count()

275701

In [20]:
##Use show() to display only the name and location (city, region, country_code) in the resulting Spark DataFrame
repeating_letter_companies.select("name", "city", "region", "country_code").show()


+--------------------+----------------+-----------+------------+
|                name|            city|     region|country_code|
+--------------------+----------------+-----------+------------+
|                Digg|        New York|   New York|         USA|
|            Facebook|      Menlo Park| California|         USA|
|               Accel|       Palo Alto| California|         USA|
|             Twitter|   San Francisco| California|         USA|
|             MeetMoi|        New York|   New York|         USA|
|               Joost|        New York|   New York|         USA|
|             AddThis|          Vienna|   Virginia|         USA|
|               Thoof|          Austin|      Texas|         USA|
|Hearst Communicat...|        New York|   New York|         USA|
|      Gannett Co Inc|          Mclean|   Virginia|         USA|
|            AllPeers|          Oxford|Oxfordshire|         GBR|
| Aggregate Knowledge|       San Mateo| California|         USA|
|              iSkoot|   